<a href="https://colab.research.google.com/github/soumik12345/DenseDepth/blob/master/notebooks/train_nyu_depth_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-f38d334a-78d2-5d49-71fe-e9bc4c149310)


In [2]:
import os
import urllib
from getpass import getpass

user = input('User name: ')
password = getpass('Password: ')
password = urllib.parse.quote(password)
repo_address = input('Repo Address: ')
branch_name = input('Branch name: ')

cmd_string = 'git clone https://{}:{}@github.com/{}.git -b {}'.format(
    user, password, repo_address, branch_name
)

os.system(cmd_string)
cmd_string, password = "", ""
%cd DenseDepth

User name: soumik12345
Password: ··········
Repo Address: soumik12345/DenseDepth
Branch name: master
/content/DenseDepth


In [3]:
!pip install -qq wandb

     |████████████████████████████████| 2.0MB 21.9MB/s 
     |████████████████████████████████| 163kB 41.6MB/s 
     |████████████████████████████████| 133kB 53.2MB/s 
     |████████████████████████████████| 102kB 14.4MB/s 
     |████████████████████████████████| 71kB 11.1MB/s 


In [1]:
%cd DenseDepth

/content/DenseDepth


In [2]:
import tensorflow as tf
from datetime import datetime
from wandb.keras import WandbCallback

from densedepth import (
    NYUDepthV2DataLoader,
    DenseDepth, DenseDepthLoss
)
from densedepth.utils import init_wandb, download_dataset

In [5]:
download_dataset(
    dataset_name='nyu_data',
    dataset_access_key='1nQd4hcsQVnX33vTv4dF2-xF_pjQ94S8P'
)

Downloading...
From: https://drive.google.com/uc?id=1nQd4hcsQVnX33vTv4dF2-xF_pjQ94S8P
To: /content/DenseDepth/nyu_data.zip
4.40GB [01:22, 53.0MB/s]


In [3]:
experiment_name = 'NYU_Depth_V2_single_image_overfit'

init_wandb(
    project_name='densedepth', entity='19soumik-rakshit96',
    experiment_name=experiment_name, wandb_api_key='cf0947ccde62903d4df0742a58b8a54ca4c11673'
)

wandb: Currently logged in as: 19soumik-rakshit96 (use `wandb login --relogin` to force relogin)


In [4]:
loader = NYUDepthV2DataLoader(
    data_dir='./data/',
    image_size=[480, 640], val_split=0.2, single_image_overfit=True
)
loader.summarize()
train_dataset, val_dataset = loader.get_datasets(batch_size=1)
print(train_dataset)
print(val_dataset)

Train RGB Images: 1
Train Depth Images: 1
validation RGB Images: 1
Validation Depth Images: 1
<PrefetchDataset shapes: ((None, 480, 640, None), (None, 240, 320, None)), types: (tf.float32, tf.float32)>
<PrefetchDataset shapes: ((None, 480, 640, None), (None, 240, 320, None)), types: (tf.float32, tf.float32)>


In [5]:
model = DenseDepth()
model.compile(
    optimizer=tf.keras.optimizers.Adam(
        learning_rate=1e-4, amsgrad=True
    ), loss=DenseDepthLoss(
        lambda_weight=0.1, depth_max_val=1000.0 / 10.0
    )
)

51879936/51877672 [==============================] - 0s 0us/step


In [6]:
callbacks = [
    tf.keras.callbacks.TensorBoard(
        log_dir='./logs/train/' + datetime.now().strftime('%Y%m%d-%H%M%S'),
        histogram_freq=1, update_freq=50, write_images=True
    ),
    WandbCallback(),
    tf.keras.callbacks.ModelCheckpoint(
        './logs/train/' + experiment_name + '_{epoch}.ckpt', save_weights_only=True
    )
]

In [7]:
history = model.fit(
    train_dataset, validation_data=val_dataset,
    epochs=15, callbacks=callbacks
)

Epoch 1/15
1/1 [==============================] - ETA: 0s - loss: 0.7831

wandb: WARNING When using several event log directories, please call wandb.tensorboard.patch(root_logdir="...") before wandb.init


1/1 [==============================] - 60s 60s/step - loss: 0.7831 - val_loss: 0.6240


wandb: ERROR Can't save model, h5py returned error: Saving the model to HDF5 format requires the model to be a Functional model or a Sequential model. It does not work for subclassed models, because such models are defined via the body of a Python method, which isn't safely serializable. Consider saving to the Tensorflow SavedModel format (by setting save_format="tf") or using `save_weights`.


Epoch 2/15
1/1 [==============================] - 3s 3s/step - loss: 0.5959 - val_loss: 0.4650
Epoch 3/15
1/1 [==============================] - 1s 581ms/step - loss: 0.3069 - val_loss: 0.3630
Epoch 4/15
1/1 [==============================] - 1s 526ms/step - loss: 0.2895 - val_loss: 0.3330
Epoch 5/15
1/1 [==============================] - 1s 558ms/step - loss: 0.2161 - val_loss: 0.3302
Epoch 6/15
1/1 [==============================] - 0s 495ms/step - loss: 0.1576 - val_loss: 0.3384
Epoch 7/15
1/1 [==============================] - 1s 512ms/step - loss: 0.1836 - val_loss: 0.3169
Epoch 8/15
1/1 [==============================] - 0s 451ms/step - loss: 0.1542 - val_loss: 0.2809
Epoch 9/15
1/1 [==============================] - 0s 459ms/step - loss: 0.1283 - val_loss: 0.2649
Epoch 10/15
1/1 [==============================] - 0s 453ms/step - loss: 0.1392 - val_loss: 0.2614
Epoch 11/15
1/1 [==============================] - 1s 511ms/step - loss: 0.1123 - val_loss: 0.2692
Epoch 12/15
1/1 [====